In [23]:
function filter_jl_files(dir_list::Vector{String})
    jl_files = filter(x -> endswith(x, ".jl"), dir_list)
    non_jl_files = setdiff(dir_list, jl_files)
    return jl_files, non_jl_files
end

filter_jl_files (generic function with 1 method)

In [45]:
function add_filepaths!(file_dict::Dict{String, String}, dir_path::String, dir_list::Vector{String})
    for file in dir_list
        file_dict[file] = joinpath(dir_path, file)
    end
end

add_filepaths! (generic function with 1 method)

In [66]:
function find_jl_files(dir_path::String)
    file_dict = Dict{String, String}()
    jl_files, non_jl_files = filter_jl_files(readdir(dir_path))
    add_filepaths!(file_dict, dir_path, jl_files)
    new_dirs = filter(x -> isdir(joinpath(dir_path, x)), non_jl_files)
    for new_dir in new_dirs
        merge!(file_dict, find_jl_files(joinpath(dir_path, new_dir)))
    end
    return file_dict
end

find_jl_files (generic function with 1 method)

In [67]:
# Get all Julia files in the src directory
scr_path = joinpath(pwd(), "src")
file_dict = find_jl_files(scr_path)

Dict{String, String} with 145 entries:
  "h2_production_commit.jl"          => "d:\\github_copies\\DOLPHYN-dev\\src\\H…
  "write_reserve_margin_revenue.jl"  => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_co2_cap.jl"                  => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "flexible_demand.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "investment_discharge.jl"          => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "write_reg.jl"                     => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "write_time_weights.jl"            => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_inputs.jl"                   => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "emissions_power.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "load_period_map.jl"               => "d:\\github_copies\\DOLPHYN-dev\\src\\G…
  "h2_pipeline.jl"                   => "d:\\github_copies\\DOLPHYN-dev\\src\\H…
  "load_h2_g2p_variability.jl"       => "d:\\github_copies\\DOLPHYN-de

In [68]:
function get_var_name(line::String)
    x = split(line, "@variable(")
    x = split(x[2], ",")
    if contains(x[2], "[")
        x = split(x[2], "[")
        return strip(x[1])
    else
        return strip(x[2])
    end
end

get_var_name (generic function with 1 method)

In [69]:
function get_exp_name(line::String)
    x = split(line, "@expression(")
    x = split(x[2], ",")
    if contains(x[2], "[")
        x = split(x[2], "[")
        return strip(x[1])
    else
        return strip(x[2])
    end
end

get_exp_name (generic function with 1 method)

In [70]:
function filter_exp_var!(exp_and_vars::Dict{String, Dict{String, String}}, line::String)
    if contains(line, "@variable")
        var_name = get_var_name(line)
        exp_and_vars["var"][var_name] = file_name
    elseif contains(line, "@expression")
        exp_name = get_exp_name(line)
        exp_and_vars["exp"][exp_name] = file_name
    end
end

filter_exp_var! (generic function with 1 method)

In [71]:
function clean_file(dir_path::String)
    open(dir_path, "r") do file
        output_lines = ""
        for line in eachline(file)
            if !startswith(line, "#") && line != ""
                output_lines = string(output_lines, strip(line))
            end
        end
        return output_lines
    end
end

clean_file (generic function with 1 method)

In [151]:
function extract_model_name(filestring::AbstractString)
    x = split(filestring, ",")
    if contains(x[2], "[")
        x = split(x[2], "[")
        return strip(x[1])
    else
        return strip(x[2])
    end
end

extract_model_name (generic function with 1 method)

In [214]:
function filter_filestring!(filestring::String, filter_string::String)
    cases = split(filestring, filter_string)
    outputs = Vector{String}()
    for i in 2:length(cases)
        push!(outputs, extract_model_name(cases[i]))
    end
    return outputs
end

filter_filestring! (generic function with 1 method)

In [243]:
function find_exp_var_origins(file_dict::Dict{String, String})
    exp_and_vars = Dict{String, Dict{String, String}}("var" => Dict{String, String}(), "exp" => Dict{String, String}())
    for (file_name, file_path) in file_dict
        # println(file_name)
        file_string = clean_file(file_path)
        new_var = filter_filestring!(file_string, "@variable")
        for var in new_var
            exp_and_vars["var"][var] = file_name
        end
        new_exp = filter_filestring!(file_string, "@expression")
        for exp in new_exp
            exp_and_vars["exp"][exp] = file_name
        end
    end
    return exp_and_vars
end

exp_and_vars = find_exp_var_origins(file_dict)

Dict{String, Dict{String, String}} with 2 entries:
  "exp" => Dict("eH2G2PCFix"=>"h2_g2p_investment.jl", "eTotalCVarFlexIn"=>"flex…
  "var" => Dict("vREG_discharge"=>"reserves.jl", "vS_HYDRO"=>"hydro_res.jl", "v…

In [244]:
exp_and_vars["var"]

Dict{String, String} with 88 entries:
  "vREG_discharge"    => "reserves.jl"
  "vS_HYDRO"          => "hydro_res.jl"
  "vRSV_discharge"    => "reserves.jl"
  "vPROD_TRANSCAP_ON" => "transmission.jl"
  "vCONTINGENCY_AUX"  => "reserves.jl"
  "vCHARGE_FLEX"      => "flexible_demand.jl"
  "vH2N_full"         => "h2_truck_all.jl"
  "vH2Ncharged"       => "h2_truck_all.jl"
  "vH2GenShut"        => "h2_production_commit.jl"
  "vH2Narrive_empty"  => "h2_truck_all.jl"
  "vH2Narrive_full"   => "h2_truck_all.jl"
  "vFLOW"             => "transmission.jl"
  "vSHUT"             => "ucommit.jl"
  "vPG2P"             => "h2_g2p_discharge.jl"
  "vRETCAPENERGY"     => "investment_energy.jl"
  "vH2TruckEnergy"    => "h2_truck_investment.jl"
  "vH2N_empty"        => "h2_truck_all.jl"
  "vH2Navail_full"    => "h2_truck_all.jl"
  "vH2Ntravel_full"   => "h2_truck_all.jl"
  ⋮                   => ⋮

In [245]:
exp_and_vars["exp"]

Dict{String, String} with 136 entries:
  "eH2G2PCFix"                 => "h2_g2p_investment.jl"
  "eTotalCVarFlexIn"           => "flexible_demand.jl"
  "eTotalCFixCharge"           => "investment_charge.jl"
  "eH2GenerationByZone"        => "h2_production.jl"
  "eTotalCVarOut"              => "discharge.jl"
  "eTotalCStartT"              => "ucommit.jl"
  "ePowerBalanceH2G2PNoCommit" => "h2_g2p_no_commit.jl"
  "eH2G2PTotalCap"             => "h2_g2p_investment.jl"
  "eH2EmissionsByPlant"        => "emissions_hsc.jl"
  "eH2GenCommit"               => "h2_production_commit.jl"
  "eCFixCharge"                => "investment_charge.jl"
  "eCVarH2Stor_in"             => "h2_storage_all.jl"
  "eTotalCVarInT"              => "storage_all.jl"
  "eTotalCFixH2TruckCharge"    => "h2_truck_investment.jl"
  "eTotalCapCharge"            => "investment_charge.jl"
  "eNet_Export_Flows"          => "transmission.jl"
  "eTotalH2GenCFix"            => "h2_investment.jl"
  "eRegReq"                    => 